<a href="https://colab.research.google.com/github/PedroVerardo/my_1_WebScraping/blob/main/my_1_WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing libraries
from urllib.request import urlopen, urlretrieve
from bs4 import BeautifulSoup
import pandas as pd

#Declaring variable cards
cards=[]

#define the url
url='https://www.pneustore.com.br/'
dados=input("(largura)/(Perfil)R(aro)")
largura,perfil_aro=dados.split("/")
urlnovo=url+"/search?q="+largura+"%2F"+perfil_aro+"%3Arelevance&page=0"

#figuring out the number of pages
response=urlopen(urlnovo)
html = response.read().decode('utf-8')
soup = BeautifulSoup(html, 'html.parser')
pag=soup.findAll("ul",{"class":"pagination"})[0].getText()
pag=pag.split("\n")
pages=len(pag[1:-1])

#making the maps
#(print (cards)) to see better
a=0
for a in range(pages+1):
  response=urlopen(urlnovo[:-1]+str(a))
  html = response.read().decode('utf-8')
  soup = BeautifulSoup(html, 'html.parser')

  
  anuncios = soup.findAll('script',{"type":"text/javascript"})[6].getText()
  anuncios = anuncios.replace('\n',"")
  anuncios = anuncios.replace('\t',"")
  anuncios = anuncios.replace(' ',"")
  anuncios = anuncios.replace('}',"")
  anuncios = anuncios.replace(']',"")
  anuncios=anuncios[92:].split("{")
  anuncios=anuncios[1:]


  i=0
  for i in range(len(anuncios)):
    if "BônusCarrinho" in anuncios[i]:
      anuncios[i]=anuncios[i].replace("'BônusCarrinhoR$40,00'","BônusCarrinho")
    anuncios2=anuncios[i].split(",")
    anuncios3=anuncios2[:8]
    for anuncio in anuncios3:
      card={}
      categoria,valor=anuncio.split(":")
      card[categoria]=valor
      cards.append(card)
    i+=1
  


#Making the DataFrame
dataset=pd.DataFrame(cards)
i=1
I=0
while I<len(dataset):
  for i in range(7):
    if i == 7:
      dataset.iat[I,i]=dataset.iat[(i+1)+I,i]
    else:
      dataset.iat[I,(i+1)]=dataset.iat[(i+1)+I,(i+1)]
  I+=8
#drop the useless information    
dataset2 = dataset.drop(columns=["'list'","'position'"])
dataset3=dataset2.dropna()
dataset3.rename(columns={"'name'":"name","'id'":"id","'price'":"price(R$)","'brand'":"brand","'category'":"category","'variant'":"variant"}, inplace=True)
dataset3.to_csv('./dataset.csv', sep=';', index = False, encoding = 'utf-8-sig')


(largura)/(Perfil)R(aro)225/60r17


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
